In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [2]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


In [15]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")



QSGD
Before Quantization:  [9.602516  4.5155435 1.8452946 2.3192837 7.6018715 9.087472  4.4516053
 8.550069  4.7978096 9.133822  9.016326  3.9488978 4.72986   9.617058
 7.335939  4.0205336]
Encoded b'$\x00\x00\x00\xb4\n\xdaA\xb2\xcadR\x00\x00\x00\x00'
After Quantization:  [13.627613  0.        0.        0.        0.        0.       13.627613
  0.        0.       13.627613  0.        0.       13.627613  0.
 13.627613  0.      ]
52.88461538461539% smaller 


In [5]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")


GZip
Before Quantization:  [8.392763  1.8799031 0.7727104 1.8461773 7.3292036 6.8138433 3.4568703
 3.5700812 7.6936374 1.4033586 2.7966292 3.9259048 6.7647533 6.18365
 9.893251  7.9981265]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncd\x00\x01F(bd\xc8\xdd\xb2\xc3\x11\x00\xe2a\x19\x17\x14\x00\x00\x00'
After Quantization:  [23.088099  0.        0.        0.        0.        0.       23.088099
  0.        0.       23.088099  0.        0.       23.088099  0.
 23.088099  0.      ]
37.5% smaller 


In [12]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")


e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.10.3+0.x64.w64.mingw32. You currently have `1.10.2+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.3+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


FP8
Before Quantization:  [4.126472  2.749015  3.8986733 9.317252  2.5157616 9.321195  8.882262
 2.9293163 9.324964  1.5345355 2.1583395 1.4662943 9.918267  6.0527186
 8.331317  7.22466  ]
Encoded b'EBCIAHIAI?@>IFHG'
After Quantization:  [ 5.    3.    3.5  10.    2.5   8.   10.    2.5  10.    1.75  2.    1.5
 10.    6.    8.    7.  ]
52.88461538461539% smaller 


In [29]:

import cv2


class customCompression():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[5.4346137  0.39908051 0.54436535 7.3529234  3.7397492  3.752178
 0.53688484 0.20883559 8.684521   9.365884   4.9378195  0.7419795
 5.63708    1.8902718  1.9684689  0.8976962 ]
Encoded [37399  1361  2401 51128 25269 25358  2347     0 60658 65535 33844  3815
 38848 12033 12593  4930]
After Quantization:  [5.43450808 0.39900485 0.54432155 7.3528283  3.7396123  3.75204806
 0.53677626 0.20883559 8.68443232 9.36588383 4.93777647 0.74189638
 5.63697338 1.89017779 1.96842525 0.89769265]
<class 'numpy.uint16'> 41.37931034482759% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  7.12347644835608e-09


In [4]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))

print(sys.getsizeof(enc))


552
552
202


unint8
36.231884057971016% smaller 
enc shape:  (2, 1, 5, 5)
dec shape:  (2, 1, 5, 5)
mse:  4.056377131919988e-06

In [2]:
from scipy.fftpack import dct, idct

class customFreq():
    def __init__(self, precision_levels = [8, 4, 2]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized

In [3]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp=cmp

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	# def get_weights(self, dtype=np.float32):
        
	# 	precision = self.precision 
	# 	weights = []
	# 	start_size = 0
	# 	end_size = 0
        
        
	# 	for layer in self.model:
			
	# 		try:
	# 			layer_name = layer.__repr__
	# 			print('Layer Name: ', layer_name)
	# 			# weights.append([np.around(layer.weight.detach().numpy().astype(dtype), decimals=precision), np.around(layer.bias.detach().numpy().astype(dtype), decimals=precision)])

    #             # layer_name = layer.__class__name__
    #             # if layer_name = 'Conv2d':
    #             #     layer_np = layer.weight.detach().numpy().astype(dtype)
    #             #     for filters in layer_np:
    #             #         for channels in filters:
    #             #             for inp_dim1 in channels:
    #             #                 fp8.encode(inp_dim1)
                
				
	# 			layer_weights = layer.weight.detach().numpy().astype(dtype)
	# 			start_size = start_size + sys.getsizeof(layer_weights)
	# 			layer_bias = layer.bias.detach().numpy().astype(dtype)
	# 			layer_weights_enc = self.cmp.encode(layer_weights.flatten())
	# 			layer_bias_enc = self.cmp.encode(layer_bias.flatten())
	# 			decoded_weights = self.cmp.decode(layer_weights_enc).reshape(layer_weights.shape)
	# 			end_size = end_size + sys.getsizeof(decoded_weights)
	# 			print("decoded_weights.shape",decoded_weights.shape)
	# 			decoded_bias = self.cmp.decode(layer_bias_enc).reshape(layer_bias.shape)
	# 			weights.append([decoded_weights,decoded_bias])
	# 		except:
	# 			continue
	# 	self.compression_ratio = ((start_size-end_size)/start_size)*100
	# 	return np.array(weights),start_size,end_size



	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size


	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights

	def client_generator(self, train_x, train_y):
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()
		
		return weights, running_loss, start_size,end_size
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight



#%%

In [5]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)
cmp = customFreq()

# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)

  0%|          | 0/164 [00:00<?, ?it/s]

layer_name <bound method Module.__repr__ of Conv2d(1, 2, kernel_size=(5, 5), stride=(1, 1))>
2
torch.Size([2, 1, 5, 5])
Parameter containing:
tensor([[[[-0.0987,  0.0324, -0.0006,  0.1326, -0.1166],
          [ 0.0617, -0.1794,  0.1544, -0.1024,  0.1977],
          [ 0.1142, -0.1490,  0.1352,  0.0402, -0.1834],
          [-0.0960, -0.0115, -0.1335,  0.1792,  0.1388],
          [ 0.0792,  0.0264,  0.1178, -0.1143, -0.1608]]],


        [[[-0.0115,  0.1428,  0.0118, -0.1396, -0.1109],
          [-0.0498, -0.0102, -0.1513,  0.0632, -0.1586],
          [ 0.1208, -0.1810, -0.0081, -0.1967, -0.1024],
          [ 0.1926, -0.1098,  0.1869,  0.1169,  0.1518],
          [-0.1553,  0.0429, -0.0674, -0.1835,  0.0529]]]], requires_grad=True)
layer_name <bound method Module.__repr__ of ReLU()>
layer_name <bound method Module.__repr__ of Conv2d(2, 4, kernel_size=(7, 7), stride=(1, 1))>
4
torch.Size([4, 2, 7, 7])
Parameter containing:
tensor([[[[-7.7785e-02,  6.7378e-02, -3.0790e-02, -1.6949e-02, -3.6

# Uint16-custom2


{'Epoch': 1, 'Loss': 2.30095}: 100%|██████████| 164/164 [00:59<00:00,  2.74it/s]
Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.09561}: 100%|██████████| 164/164 [00:57<00:00,  2.84it/s]
Test Accuracy: 0.36056
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.86639}: 100%|██████████| 164/164 [00:58<00:00,  2.81it/s]
Test Accuracy: 0.63837
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.77721}: 100%|██████████| 164/164 [00:56<00:00,  2.90it/s]
Test Accuracy: 0.68152
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.74531}: 100%|██████████| 164/164 [01:01<00:00,  2.68it/s]
Test Accuracy: 0.71413
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.72985}: 100%|██████████| 164/164 [00:44<00:00,  3.71it/s]
Test Accuracy: 0.73011
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.64313}: 100%|██████████| 164/164 [01:02<00:00,  2.63it/s]
Test Accuracy: 0.73162
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54832}: 100%|██████████| 164/164 [00:42<00:00,  3.83it/s]
Test Accuracy: 0.90883
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53205}: 100%|██████████| 164/164 [00:22<00:00,  7.37it/s]
Test Accuracy: 0.92051
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.52779}: 100%|██████████| 164/164 [00:40<00:00,  4.01it/s]
Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784

# uint8-custom 1
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:54<00:00,  3.00it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.13939}: 100%|██████████| 164/164 [00:44<00:00,  3.65it/s]
Test Accuracy: 0.38529
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.98048}: 100%|██████████| 164/164 [00:23<00:00,  6.95it/s]
Test Accuracy: 0.44234
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.82871}: 100%|██████████| 164/164 [00:43<00:00,  3.76it/s]
Test Accuracy: 0.64131
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.77463}: 100%|██████████| 164/164 [00:25<00:00,  6.49it/s]
Test Accuracy: 0.68517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.75027}: 100%|██████████| 164/164 [00:24<00:00,  6.64it/s]
Test Accuracy: 0.70226
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.73521}: 100%|██████████| 164/164 [00:47<00:00,  3.49it/s]
Test Accuracy: 0.71979
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.71573}: 100%|██████████| 164/164 [00:44<00:00,  3.67it/s]
Test Accuracy: 0.73043
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.68576}: 100%|██████████| 164/164 [00:53<00:00,  3.06it/s]
Test Accuracy: 0.73663
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.64119}: 100%|██████████| 164/164 [00:21<00:00,  7.65it/s]
Test Accuracy: 0.8215
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  399.71174716949463